# IMDB Demo

In [14]:
using Knet,WordTokenizers
BATCHSIZE=64
SEED=1311194
MAXLEN=150 #maximum size of the word sequence
MAXFEATURES=30000 #vocabulary size
gpu(0)
atype = gpu()<0 ? Array{Float32}:KnetArray{Float32}

Knet.KnetArray{Float32,N} where N

In [2]:
function loadmodel(url="ai.ku.edu.tr/models/model_imdb.jld",localfile="model_imdb.jld")
    !isfile(localfile) && download(url,localfile)
    d = load(localfile)
    weights = d["weights"];rnnSpec=d["rnnSpec"];
    return weights,rnnSpec
end

loadmodel (generic function with 3 methods)

In [3]:
function predict(weights, inputs, rnnSpec;train=false)
    rnnWeights, inputMatrix, outputMatrix = weights # (1,1,W), (X,V), (2,H)
    indices = hcat(inputs...)' # (B,T)
    rnnInput = inputMatrix[:,indices] # (X,B,T)
    rnnOutput = rnnforw(rnnSpec, rnnWeights, rnnInput)[1] # (H,B,T)
    return outputMatrix * rnnOutput[:,:,end] # (2,H) * (H,B) = (2,B)
end

predict (generic function with 1 method)

In [6]:
include("imdb.jl")
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=MAXFEATURES,seed=SEED)
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end
imdbarray = Array{String}(88584)
for (k,v) in imdbdict; imdbarray[v]=k; end

INFO: Loading IMDB...


 15.714042 seconds (16.02 M allocations: 810.981 MiB, 4.61% gc time)
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}


In [7]:
rnd = rand(1:length(xtrn))
println("Sample review:\n",join(imdbarray[xtrn[rnd]]," "),"\n")
println("Classification: ",join(ytrn[rnd]))

Sample review:
reilly reilly reilly reilly reilly reilly reilly 5hrs to start with i have to point out the fact that you're gonna feel completely lost for more than half an hour yeah some things happen but you don't know why or what for when you finally figure things out you just realize that it's nothing but a twisted spastic opera dealing with mature prostitutes dead mothers illegitimate sons the characters are rather poor and the actors specially the young ones don't help that much to spastic look credible only marisa spastic stands out but she's a superb actress no matter if the movie is pure rubbish br br the only positive things to say about spastic sol de spastic is that spastic pablo spastic seems to have good intentions and he's filmed a couple of scenes that are quite intense well maybe the next time br br my rate 4 10

Classification: 1


In [17]:
weights,rnnSpec = loadmodel()

((Knet.KnetArray{Float32,3}(Knet.KnetPtr(Ptr{Void} @0x000001040f4a6e00, 363200, 0, nothing), (1, 1, 90800)), Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x0000010410c00000, 15000000, 0, nothing), (125, 30000)), Knet.KnetArray{Float32,2}(Knet.KnetPtr(Ptr{Void} @0x000001040f600a00, 800, 0, nothing), (2, 100))), Knet.RNN(125, 100, 1, 0.0, 0, 0, 2, 0, Float32, Knet.RD(Ptr{Void} @0x00007fab7f745990), Knet.DD(Ptr{Void} @0x00007fab7f0066d0, Knet.KnetArray{UInt8,1}(Knet.KnetPtr(Ptr{Void} @0x000001040fc93e00, 1474560, 0, nothing), (1474560,))), nothing, nothing, nothing))

In [18]:
if !(typeof(first(weights)) <: atype)
    weights= map(atype,weights)
end

In [19]:
info("Testing...")
@time accuracy(weights, minibatch(xtst,ytst,BATCHSIZE), (w,x)->predict(w,x,rnnSpec))

INFO: Testing...


  4.388864 seconds (102.95 k allocations: 36.532 MiB, 0.66% gc time)


0.8670272435897436

In [20]:
userinput = readline(STDIN)
words = tokenize(lowercase(userinput))
ex = [get!(imdbdict,wr,length(imdbdict)) for wr in words]

STDIN> it was such a bad movie


6-element Array{Int32,1}:
   9
  13
 138
   3
  75
  17

In [21]:
y = predict(weights,[ex],rnnSpec;train=false)
category = indmax(Array(y))

1